In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from pathlib import Path
from glob import glob
import open3d as o3d
import numpy as np

BASE_PATH = Path("/home/SE3D/SE3D/datasets/scannet/")

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [13]:
scan_paths = glob(str(BASE_PATH) + "/scans/scene*/*_vh_clean_2.labels.ply")

scan_path = scan_paths[0]

mesh = o3d.io.read_triangle_mesh(scan_path)
print(mesh)


TriangleMesh with 81369 points and 153587 triangles.


In [11]:
mesh_vertices = np.asarray(mesh.vertices)

In [12]:
print(mesh_vertices.shape)

(81369, 3)


In [21]:
# Load segments file
segments_path = ".".join(scan_path.split(".")[:-2]) + ".0.010000.segs.json"
segment_class_path = "_".join(scan_path.split("_")[:-1]) + ".aggregation.json"

In [23]:
import json
with open(segments_path) as f:
    segments_data = json.load(f)
with open(segment_class_path) as f:
    segments_class_data = json.load(f)

In [ ]:
# This indicates the segment to which each vertex belongs to
vertex_segment_assignment = segments_data["segIndices"]  
vertex_segment_assignment

In [ ]:
# This is a list of dictionaries, each of which contains "segments" which is a list of segment ids from "segIndices" above
# and a "label" which identifies the class of the object.
segment_merge_class_assignment = segments_class_data["segGroups"]
segment_merge_class_assignment

In [32]:
# Find all classes in the scan
classes = set()
for merged_segment in segment_merge_class_assignment:
    classes.update([merged_segment["label"]])
print(classes)

{'clock', 'trash can', 'pillow', 'toaster oven', 'guitar', 'bed', 'nightstand', 'microwave', 'dish rack', 'guitar case', 'kitchen cabinets', 'table', 'shoes', 'doorframe', 'scale', 'kitchen counter', 'door', 'tissue box', 'mirror', 'shower', 'wall', 'toilet', 'toaster', 'backpack', 'laundry basket', 'floor', 'object', 'curtain', 'ceiling', 'couch', 'shelf', 'coffee table', 'cabinet', 'refrigerator', 'bicycle', 'window', 'stool', 'tv', 'desk', 'sink'}


In [43]:
# Create class ids
class_list = list(classes)
class_ids = {}
for idx, classname in enumerate(class_list):
    class_ids[classname] = idx
class_ids

{'clock': 0,
 'trash can': 1,
 'pillow': 2,
 'toaster oven': 3,
 'guitar': 4,
 'bed': 5,
 'nightstand': 6,
 'microwave': 7,
 'dish rack': 8,
 'guitar case': 9,
 'kitchen cabinets': 10,
 'table': 11,
 'shoes': 12,
 'doorframe': 13,
 'scale': 14,
 'kitchen counter': 15,
 'door': 16,
 'tissue box': 17,
 'mirror': 18,
 'shower': 19,
 'wall': 20,
 'toilet': 21,
 'toaster': 22,
 'backpack': 23,
 'laundry basket': 24,
 'floor': 25,
 'object': 26,
 'curtain': 27,
 'ceiling': 28,
 'couch': 29,
 'shelf': 30,
 'coffee table': 31,
 'cabinet': 32,
 'refrigerator': 33,
 'bicycle': 34,
 'window': 35,
 'stool': 36,
 'tv': 37,
 'desk': 38,
 'sink': 39}

In [44]:
# Assign a class to each segment
segment_classes = {}
for merged_segment in segment_merge_class_assignment:
    for segment in merged_segment["segments"]:
        segment_classes[segment] = merged_segment["label"]
segment_classes

{43652: 'window',
 43832: 'window',
 43632: 'window',
 53294: 'window',
 44062: 'window',
 44013: 'window',
 44158: 'window',
 53070: 'window',
 53173: 'window',
 53253: 'window',
 77796: 'window',
 77832: 'window',
 78722: 'window',
 42731: 'table',
 42300: 'table',
 43161: 'table',
 40270: 'table',
 41048: 'table',
 41919: 'table',
 50223: 'table',
 49593: 'table',
 49628: 'table',
 51919: 'table',
 48329: 'table',
 38790: 'table',
 50194: 'table',
 39867: 'table',
 40672: 'table',
 41692: 'table',
 49958: 'table',
 40054: 'table',
 40065: 'table',
 40400: 'table',
 41630: 'table',
 51890: 'table',
 41970: 'table',
 42213: 'table',
 43106: 'table',
 43164: 'table',
 49651: 'table',
 49813: 'table',
 50107: 'table',
 50313: 'table',
 67857: 'kitchen counter',
 68082: 'kitchen counter',
 68224: 'kitchen counter',
 70807: 'shower',
 70410: 'shower',
 71001: 'shower',
 70733: 'shower',
 71261: 'shower',
 70131: 'shower',
 70098: 'shower',
 70528: 'shower',
 70206: 'shower',
 55339: 'show

In [70]:
# Assign a class to each vertex

vertices_classes = [class_ids[segment_classes[e]] if e in segment_classes.keys() else -1 for e in vertex_segment_assignment]
vertices_classes = np.array(vertices_classes)
vertices_classes

array([27, 27, 27, ..., 20, 20, 20])

In [ ]:
import numpy as np
from open3d.web_visualizer import draw
from matplotlib import pyplot as plt

# Assuming you have a list of 3D points and a list of corresponding classes
# Replace these lists with your actual data

# Create Open3D point cloud
point_cloud = o3d.geometry.PointCloud()
point_cloud.points = o3d.utility.Vector3dVector(mesh_vertices)

# Assign colors based on classes
# color_map = o3d.visualization.draw_geometries.create_color_map(vertices_classes, max_classes=40)
colors = plt.cm.jet(vertices_classes / 40.0)[:, :3]
point_cloud.colors = o3d.utility.Vector3dVector(colors)

# Visualize the point cloud
draw(point_cloud)

[5092:308][1795719] (stun_port.cc:96): Binding request timed out from 10.79.23.x:42349 (eno1)


In [ ]:
voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(point_cloud,
                                                            voxel_size=0.05)
draw([voxel_grid])